In [2]:
import sys
sys.path.append("./..")


import pandas as pd
import numpy as np
import preprocessing as prep
from calanderific_holiday_api import *
import pickle
from datetime import datetime, timedelta, date, time

In [3]:
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None


In [4]:
df = pd.read_csv("../Data/new_york/year_lga_dep.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 52,DAY_OF_YEAR
0,495,450445,2018,2,6,1,5,2018-06-01,UA,N821UA,761,31703,LGA,"New York, NY",NY,30977,ORD,"Chicago, IL",IL,545,706.0,81.0,81.0,1.0,5.0,0001-0559,16.0,722.0,809.0,11.0,706,820.0,74.0,74.0,1.0,4.0,0700-0759,0.0,NaN,0.0,141.0,134.0,107.0,1.0,733.0,3,74.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,6_1
1,557,359002,2018,2,6,1,5,2018-06-01,AA,N986NN,1283,31703,LGA,"New York, NY",NY,32467,MIA,"Miami, FL",FL,550,548.0,-2.0,0.0,0.0,-1.0,0001-0559,19.0,607.0,833.0,5.0,855,838.0,-17.0,0.0,0.0,-2.0,0800-0859,0.0,NaN,0.0,185.0,170.0,146.0,1.0,1096.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1
2,602,354259,2018,2,6,1,5,2018-06-01,AA,N919AN,1111,31703,LGA,"New York, NY",NY,30194,DFW,"Dallas/Fort Worth, TX",TX,555,554.0,-1.0,0.0,0.0,-1.0,0001-0559,22.0,616.0,845.0,11.0,849,856.0,7.0,7.0,0.0,0.0,0800-0859,0.0,NaN,0.0,234.0,242.0,209.0,1.0,1389.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1
3,744,174928,2018,2,6,1,5,2018-06-01,WN,N481WN,1821,31703,LGA,"New York, NY",NY,30194,DAL,"Dallas, TX",TX,600,614.0,14.0,14.0,0.0,0.0,0600-0659,24.0,638.0,839.0,6.0,850,845.0,-5.0,0.0,0.0,-1.0,0800-0859,0.0,NaN,0.0,230.0,211.0,181.0,1.0,1381.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1
4,745,174933,2018,2,6,1,5,2018-06-01,WN,N227WN,1888,31703,LGA,"New York, NY",NY,31453,HOU,"Houston, TX",TX,600,557.0,-3.0,0.0,0.0,-1.0,0600-0659,18.0,615.0,816.0,12.0,900,828.0,-32.0,0.0,0.0,-2.0,0900-0959,0.0,NaN,0.0,240.0,211.0,181.0,1.0,1428.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172073 entries, 0 to 172072
Data columns (total 56 columns):
Unnamed: 0               172073 non-null int64
Unnamed: 0.1             172073 non-null int64
YEAR                     172073 non-null int64
QUARTER                  172073 non-null int64
MONTH                    172073 non-null int64
DAY_OF_MONTH             172073 non-null int64
DAY_OF_WEEK              172073 non-null int64
FL_DATE                  172073 non-null object
OP_UNIQUE_CARRIER        172073 non-null object
TAIL_NUM                 171761 non-null object
OP_CARRIER_FL_NUM        172073 non-null int64
ORIGIN_CITY_MARKET_ID    172073 non-null int64
ORIGIN                   172073 non-null object
ORIGIN_CITY_NAME         172073 non-null object
ORIGIN_STATE_ABR         172073 non-null object
DEST_CITY_MARKET_ID      172073 non-null int64
DEST                     172073 non-null object
DEST_CITY_NAME           172073 non-null object
DEST_STATE_ABR           172073 non

In [6]:
df.describe()

,Unnamed: 0,Unnamed: 0.1,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 52
count,1.720730e+05,172073.000000,172073.000000,172073.000000,172073.000000,172073.000000,172073.000000,172073.000000,172073.0,172073.000000,172073.000000,166999.000000,166735.000000,166735.000000,166735.000000,166735.000000,166651.000000,166651.000000,166614.000000,166614.000000,172073.000000,166615.000000,166110.000000,166110.000000,166110.000000,166110.000000,172073.000000,172073.000000,172073.000000,166207.000000,166207.000000,172073.0,172073.000000,172073.000000,39812.000000,39812.000000,39812.000000,39812.000000,39812.000000,1889.000000,1889.000000,1889.000000,0.0
mean,3.653306e+06,312634.846135,2018.413162,2.504809,6.507128,15.791031,3.799945,3237.663823,31703.0,31666.557490,1339.573414,1344.127174,13.802693,17.360914,0.217885,0.273266,26.860721,1387.244427,1490.156956,8.043346,1540.879830,1497.368388,6.997080,18.288345,0.239673,-0.017988,0.031574,0.002516,150.992939,144.654990,109.774667,1.0,709.189507,3.335945,15.498945,3.240530,19.234904,0.085477,34.635160,1401.274749,69.053467,67.087877,NaN
std,2.101943e+06,178263.779333,0.492403,1.106298,3.396697,8.709649,1.954563,1842.033333,0.0,1335.529627,454.366920,475.846695,51.872259,50.524900,0.412810,2.611415,14.753318,478.374521,503.399641,6.381013,466.201595,507.459304,55.670815,50.344318,0.426885,2.840551,0.174863,0.050100,50.942714,52.568783,50.143947,0.0,377.955225,1.564439,47.852062,29.873242,42.810001,4.779197,55.613085,449.460390,55.329558,52.313366,NaN
min,4.950000e+02,6.000000,2018.000000,1.000000,1.000000,1.000000,1.000000,16.000000,31703.0,30154.000000,530.000000,1.000000,-40.000000,0.000000,0.000000,-2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-73.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,24.000000,42.000000,18.000000,1.0,96.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,1.000000,1.000000,NaN
25%,1.839402e+06,159340.000000,2018.000000,2.000000,4.000000,8.000000,2.000000,1632.000000,31703.0,30721.000000,936.000000,936.000000,-6.000000,0.000000,0.000000,-1.000000,17.000000,1006.000000,1118.000000,4.000000,1145.000000,1122.000000,-20.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,109.000000,103.000000,69.000000,1.0,419.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1055.000000,24.000000,24.000000,NaN
50%,3.645448e+06,324872.000000,2018.000000,3.000000,7.000000,16.000000,4.000000,3433.000000,31703.0,31057.000000,1330.000000,1331.000000,-3.000000,0.000000,0.000000,-1.000000,23.000000,1355.000000,1504.000000,6.000000,1529.000000,1509.000000,-8.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,151.000000,141.000000,108.000000,1.0,733.000000,3.000000,0.000000,0.000000,7.000000,0.000000,14.000000,1439.000000,50.000000,50.000000,NaN
75%,5.471181e+06,456131.000000,2019.000000,3.000000,9.000000,23.000000,5.000000,4759.000000,31703.0,32467.000000,1729.000000,1742.000000,9.000000,9.000000,0.000000,0.000000,33.000000,1806.000000,1916.000000,9.000000,1933.000000,1921.000000,13.000000,13.000000,0.000000,0.000000,0.000000,0.000000,184.000000,176.000000,141.000000,1.0,1010.000000,5.000000,12.000000,0.000000,23.000000,0.000000,48.000000,1734.000000,111.000000,110.000000,NaN
max,7.295516e+06,645141.000000,2019.000000,4.000000,12.000000,31.000000,7.000000,7881.000000,31703.0,35412.000000,2257.000000,2400.000000,1556.000000,1556.000000,1.000000,12.000000,169.000000,2400.000000,2400.000000,229.000000,2400.000000,2400.000000,1577.000000,1577.000000,1.000000,12.000000,1.000000,1.00

In [7]:
prep.print_df_metadata(df)

AttributeError: module 'preprocessing' has no attribute 'print_df_metadata'

In [8]:
with open('../Data/HolidayData_NY_2014-2019.bin','rb') as f:
    holiday_data = pickle.load(f)
holiday_data

{datetime.date(2014, 1, 1): {"New Year's Day": (name: New Year's Day, description: New Year's Day is the first day of the Gregorian calendar, which is widely used in many countries such as the USA., locations: ['All'], date: 2014-01-01, types: [<HolidayType.NationalHoliday: 'National holiday'>])},
 datetime.date(2014, 1, 4): {'World Braille Day': (name: World Braille Day, description: World Braille Day celebrates the life and achievements of Louis Braille, who invented the braille code for the visually impaired., locations: ['All'], date: 2014-01-04, types: [<HolidayType.WorldwideObservance: 'Worldwide observance'>])},
 datetime.date(2014, 1, 6): {'Epiphany': (name: Epiphany, description: Many people in the United States annually observe Epiphany, or Three Kings’ Day, on January 6. It is a Christian observance and a public holiday in the US Virgin Islands., locations: ['All'], date: 2014-01-06, types: [<HolidayType.Christian: 'Christian'>])},
 datetime.date(2014, 1, 7): {'Orthodox Chri

In [12]:
holiday_data[date(2017, 1, 1)]

{'Last Day of Chanukah': (name: Last Day of Chanukah, description: The last day of Hanukkah marks the end of a festive Jewish occasion, known as Hanukkah or Chanukah., locations: ['All'], date: 2017-01-01, types: [<HolidayType.Hebrew: 'Hebrew'>]),
 "New Year's Day": (name: New Year's Day, description: New Year's Day is the first day of the Gregorian calendar, which is widely used in many countries such as the USA., locations: ['All'], date: 2017-01-01, types: [<HolidayType.NationalHoliday: 'National holiday'>])}

In [18]:
def get_holiday_vicinity_status_per_date(holiday_data, holiday_type, start_date, 
                                         end_date, vicinity = 3):
    remaining_forward_vicinity = 0
    
    ret = {}
    date = start_date 
    while date <= end_date:
        has_relevant_holiday = False
        if date in holiday_data:
            for h_name in holiday_data[date]:
                for t in holiday_data[date][h_name].types:
                    if t == holiday_type:
                        has_relevant_holiday = True
                        break
        
        if has_relevant_holiday:
            # Occurs when not in another holiday's vicinity
            if remaining_forward_vicinity == 0: 
                # remaining_backward_vicinity = vicinity
                for i in range(1, vicinity + 1):
                    ret[date - timedelta(days=i)] = True
            remaining_forward_vicinity = vicinity + 1
        
        ret[date] = remaining_forward_vicinity > 0
        remaining_forward_vicinity -= 1
        date += timedelta(days=1)
        
    return ret


In [19]:
def get_holiday_attribute_function(holiday_type):
    start = date.fromisoformat(df['FL_DATE'].min())
    end = date.fromisoformat(df['FL_DATE'].max())
    holiday_vicinity_dict = get_holiday_vicinity_status_per_date(holiday_data, holiday_type, 
                                                                 start, end)
    
    def get_attribute_by_time(df):
        dt = date.fromisoformat(df['FL_DATE'])
        return dt in holiday_vicinity_dict and holiday_vicinity_dict[dt]
    
    return get_attribute_by_time

for ht in [HolidayType.NationalHoliday, HolidayType.Christian]:
    df[ht.value.upper().replace(' ', '_')] = \
        df.apply(get_holiday_attribute_function(ht), axis = 1)

In [20]:
display(df)

,Unnamed: 0,Unnamed: 0.1,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 52,DAY_OF_YEAR,NATIONAL_HOLIDAY,CHRISTIAN
0,495,450445,2018,2,6,1,5,2018-06-01,UA,N821UA,761,31703,LGA,"New York, NY",NY,30977,ORD,"Chicago, IL",IL,545,706.0,81.0,81.0,1.0,5.0,0001-0559,16.0,722.0,809.0,11.0,706,820.0,74.0,74.0,1.0,4.0,0700-0759,0.0,NaN,0.0,141.0,134.0,107.0,1.0,733.0,3,74.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,6_1,False,False
1,557,359002,2018,2,6,1,5,2018-06-01,AA,N986NN,1283,31703,LGA,"New York, NY",NY,32467,MIA,"Miami, FL",FL,550,548.0,-2.0,0.0,0.0,-1.0,0001-0559,19.0,607.0,833.0,5.0,855,838.0,-17.0,0.0,0.0,-2.0,0800-0859,0.0,NaN,0.0,185.0,170.0,146.0,1.0,1096.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1,False,False
2,602,354259,2018,2,6,1,5,2018-06-01,AA,N919AN,1111,31703,LGA,"New York, NY",NY,30194,DFW,"Dallas/Fort Worth, TX",TX,555,554.0,-1.0,0.0,0.0,-1.0,0001-0559,22.0,616.0,845.0,11.0,849,856.0,7.0,7.0,0.0,0.0,0800-0859,0.0,NaN,0.0,234.0,242.0,209.0,1.0,1389.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1,False,False
3,744,174928,2018,2,6,1,5,2018-06-01,WN,N481WN,1821,31703,LGA,"New York, NY",NY,30194,DAL,"Dallas, TX",TX,600,614.0,14.0,14.0,0.0,0.0,0600-0659,24.0,638.0,839.0,6.0,850,845.0,-5.0,0.0,0.0,-1.0,0800-0859,0.0,NaN,0.0,230.0,211.0,181.0,1.0,1381.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1,False,False
4,745,174933,2018,2,6,1,5,2018-06-01,WN,N227WN,1888,31703,LGA,"New York, NY",NY,31453,HOU,"Houston, TX",TX,600,557.0,-3.0,0.0,0.0,-1.0,0600-0659,18.0,615.0,816.0,12.0,900,828.0,-32.0,0.0,0.0,-2.0,0900-0959,0.0,NaN,0.0,240.0,211.0,181.0,1.0,1428.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1,False,False
5,805,344569,2018,2,6,1,5,2018-06-01,AA,N952UW,1841,31703,LGA,"New York, NY",NY,34100,PHL,"Philadelphia, PA",PA,600,602.0,2.0,2.0,0.0,0.0,0600-0659,22.0,624.0,710.0,10.0,704,720.0,16.0,16.0,1.0,1.0,0700-0759,0.0,NaN,0.0,64.0,78.0,46.0,1.0,96.0,1,2.0,0.0,14.0,0.0,0.0,NaN,NaN,NaN,NaN,6_1,False,False
6,819,361874,2018,2,6,1,5,2018-06-01,AA,N768US,2126,31703,LGA,"New York, NY",NY,30721,BOS,"Boston, MA",MA,600,NaN,NaN,NaN,NaN,NaN,0600-0659,NaN,NaN,NaN,NaN,701,NaN,NaN,NaN,NaN,NaN,0700-0759,1.0,C,0.0,61.0,NaN,NaN,1.0,184.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1,False,False
7,835,389712,2018,2,6,1,5,2018-06-01,YX,N437YX,4499,31703,LGA,"New York, NY",NY,31066,CMH,"Columbus, OH",OH,600,554.0,-6.0,0.0,0.0,-1.0,0600-0659,15.0,609.0,721.0,8.0,746,729.0,-17.0,0.0,0.0,-2.0,0700-0759,0.0,NaN,0.0,106.0,95.0,72.0,1.0,479.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1,False,False
8,847,397647,2018,2,6,1,5,2018-06-01,YX,N404YX,4750,31703,LGA,"New York, NY",NY,30852,DCA,"Washington, DC",VA,600,558.0,-2.0,0.0,0.0,-1.0,0600-0659,24.0,622.0,710.0,18.0,719,728.0,9.0,9.0,0.0,0.0,0700-0759,0.0,NaN,0.0,79.0,90.0,48.0,1.0,214.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1,False,False
9,853,398365,2018,2,6,1,5,2018-06-01,YX,N214JQ,6046,31703,LGA,"New York, NY",NY,30721,BOS,"Boston, MA",MA,600,554.0,-6.0,0.0,0.0,-1.0,0600-0659,33.0,627.0,708.0,14.0,708,722.0,14.0,14.0,0.0,0.0,0700-0759,0.0,NaN,0.0,68.0,88.0,41.0,1.0,184.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_1,False,False


In [21]:
pd.options.display.max_rows = 999
print(df.isna().sum())

Unnamed: 0                    0
Unnamed: 0.1                  0
YEAR                          0
QUARTER                       0
MONTH                         0
DAY_OF_MONTH                  0
DAY_OF_WEEK                   0
FL_DATE                       0
OP_UNIQUE_CARRIER             0
TAIL_NUM                    312
OP_CARRIER_FL_NUM             0
ORIGIN_CITY_MARKET_ID         0
ORIGIN                        0
ORIGIN_CITY_NAME              0
ORIGIN_STATE_ABR              0
DEST_CITY_MARKET_ID           0
DEST                          0
DEST_CITY_NAME                0
DEST_STATE_ABR                0
CRS_DEP_TIME                  0
DEP_TIME                   5074
DEP_DELAY                  5338
DEP_DELAY_NEW              5338
DEP_DEL15                  5338
DEP_DELAY_GROUP            5338
DEP_TIME_BLK                  0
TAXI_OUT                   5422
WHEELS_OFF                 5422
WHEELS_ON                  5459
TAXI_IN                    5459
CRS_ARR_TIME                  0
ARR_TIME

In [22]:
df.to_csv('../Data/new_york/year_lga_dep_holiday.csv')